# DataFrame IV

Los objetivos de aprendizaje son:

1. Joining DataFrames
    + pd.concat()
    + merge()
2. Exportar Datos
    + CSV
    + Excel     


## 1. Joining DataFrames

En ocasiones nos interesa juntar los valores de dos `DataFrames`, Tenemos dos opciones.

### pd.concat()

Tal y como su nombre nos lo indica, la función [`pd.concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) sirve para concatenar dos objetos tales como `Series` o `DataFrame`. Sus parámetros más importantes son:

* `objs`: son los `DataFrame`s o `Series` que concatenaremos, los deberemos pasar en formato de lista.
* `axis`: Indicamos si la concatenación se hará mediante renglones o columnas.



In [1]:
import numpy as np
import pandas as pd

week1 = pd.read_csv("./Data/pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("./Data/pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("./Data/pandas/Restaurant - Customers.csv")
foods = pd.read_csv("./Data/pandas/Restaurant - Foods.csv")

In [2]:
display(week1.head())
print("Ordenes de la primera semana: {}".format(week1.shape[0]))
display(week2.head())
print("Ordenes de la segunda semana: {}".format(week2.shape[0]))

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


Ordenes de la primera semana: 250


,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


Ordenes de la segunda semana: 250


In [3]:
weeks = pd.concat(
    objs = [week1, week2],
    axis = 0
)

In [5]:
weeks.shape

(500, 2)

In [4]:
display(weeks.head())
print("Las ordenes totales son: {}".format(weeks.shape[0]))
display(weeks.tail())

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


Las ordenes totales son: 500


,Customer ID,Food ID
245,783,10
246,556,10
247,547,9
248,252,9
249,249,6


¿Qué ha sucedido con los índices?

In [6]:
weeks.loc[1]

,Customer ID,Food ID
1,97,4
1,813,7


Se han duplicado. Para resolverlo conservando los valores originales de los índices, podemos añadir un segundo índice:

In [7]:
weeks =pd.concat([week1, week2], keys = ["w1", "w2"])
weeks

Customer ID  Food ID
w1 0            537        9
   1             97        4
   2            658        1
   3            202        2
   4            155        9
...             ...      ...
w2 245          783       10
   246          556       10
   247          547        9
   248          252        9
   249          249        6

[500 rows x 2 columns]

In [9]:
list(weeks.index)[0]

('w1', 0)

O reiniciar los índices una vez concatenados los datos:

In [10]:
weeks = pd.concat([week1, week2], ignore_index = True)
weeks

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [11]:
pd.concat(
    objs = [week1, week2],
    axis = 1
)

,Customer ID,Food ID,Customer ID,Food ID
0,537,9,688,10
1,97,4,813,7
2,658,1,495,10
3,202,2,189,5
4,155,9,267,3
...,...,...,...,...
245,413,9,783,10
246,926,6,556,10
247,134,3,547,9
248,396,6,252,9


## Merge

El método [`.merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) nos da la oportunidad de unir dos objetos de las clases `DataFrame` o `Series` respetando alguna condición de igualdad entre sus columnas. Para aquellos que han programado previamente en SQL, `.merge()` es el equivalente al `Left, right, inner, outer - join`.

Veamos un ejemplo:

El `DataFrame` week1 contiene órdenes por cliente. Tanto los clientes `Customer ID` como las comidas `Food ID` están en formato numérico.


In [12]:
week1.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [13]:
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [18]:
week1.merge( 
    right=customers[["ID", "Occupation"]],
    how = "inner",
    left_on = ["Customer ID"],
    right_on = ['ID']
).drop(
    columns=["ID"]
)

,Customer ID,Food ID,Occupation
0,537,9,Registered Nurse
1,97,4,Account Coordinator
2,658,1,Community Outreach Specialist
3,202,2,Account Representative III
4,155,9,Database Administrator III
...,...,...,...
245,413,9,Technical Writer
246,926,6,Legal Assistant
247,134,3,Financial Advisor
248,396,6,Analyst Programmer


## 2. Exportar Datos

Supongamos que queremos guardar en disco el siguiente `DataFrame`:

In [21]:
week1_customer = week1.merge( 
    customers[["ID", "First Name"]],
    how = "left",
    left_on = ["Customer ID"],
    right_on = ['ID']).drop(['ID', 'Customer ID'], axis = 1)
week1_customer.head(1)

,Food ID,First Name
0,9,Cheryl


### csv 

Para guardarlo en formato csv se puede utilizar el método [`.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html)

In [23]:
import os

In [24]:
os.getcwd()

'/Users/heber.trujillo/projects/curso-python-cac/11 Data Análisis - Pandas'

In [22]:
week1_customer.to_csv(
    path_or_buf = "./Data/pandas/week1_customer.csv"
)

Veamos cómo ha quedado:

In [25]:
pd.read_csv("./Data/pandas/week1_customer.csv").head()

,Unnamed: 0,Food ID,First Name
0,0,9,Cheryl
1,1,4,Amanda
2,2,1,Patrick
3,3,2,Louis
4,4,9,Carolyn


In [26]:
week1_customer.to_csv(
    path_or_buf = "./Data/pandas/week1_customer_v2.csv", 
    index = False
)

In [27]:
pd.read_csv("./Data/pandas/week1_customer_v2.csv").head()

,Food ID,First Name
0,9,Cheryl
1,4,Amanda
2,1,Patrick
3,2,Louis
4,9,Carolyn


### excel

Excel es un formato muy amigable para casi todos los usuarios. Veamos cómo exportar un DataFrame en formato excel con el método [`.to_excel()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html) 

In [28]:
week1_customer.to_excel(
    "./Data/pandas/week1_customer.xlsx",
    sheet_name = "week1",
    index = False
)

In [29]:
pd.read_excel("./Data/pandas/week1_customer.xlsx").head()

,Food ID,First Name
0,9,Cheryl
1,4,Amanda
2,1,Patrick
3,2,Louis
4,9,Carolyn
